In [9]:
from datetime import datetime
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
from IPython.display import display
import json
import os
import tkinter as tk
from tkinter import ttk, messagebox

# # --------------------- GUI ---------------------
# Crear ventana principal
ventana = tk.Tk()
ventana.title("Control de Gastos")
ventana.geometry("500x400")

# Título
titulo = tk.Label(ventana, text="Control de Gastos Personales", font=("Arial", 16))
titulo.pack(pady=10)

# Funciones de los botones
def registrar_ingreso():
    ingreso_ventana = tk.Toplevel(ventana)
    ingreso_ventana.title("Registrar Ingreso")
    ingreso_ventana.geometry("350x300")

    # Monto
    tk.Label(ingreso_ventana, text="Monto ($):").pack(pady=5)
    entrada_monto = tk.Entry(ingreso_ventana)
    entrada_monto.pack()

    # Descripción
    tk.Label(ingreso_ventana, text="Descripción:").pack(pady=5)
    entrada_descripcion = tk.Entry(ingreso_ventana)
    entrada_descripcion.pack()

    # Categoría
    tk.Label(ingreso_ventana, text="Categoría:").pack(pady=5)
    combo_categoria = ttk.Combobox(ingreso_ventana, values=categorias, state="readonly")
    combo_categoria.pack()

    # Fecha
    tk.Label(ingreso_ventana, text="Fecha (dd-mm-yyyy) [opcional]:").pack(pady=5)
    entrada_fecha = tk.Entry(ingreso_ventana)
    entrada_fecha.pack()
    
    # Botón para guardar
    def guardar_ingreso():
        try:
            monto = float(entrada_monto.get())
            descripcion = entrada_descripcion.get()
            categoria = combo_categoria.get()
            fecha = entrada_fecha.get().strip()

            if not descripcion or not categoria:
                messagebox.showwarning("Campos incompletos", "Por favor completa todos los campos obligatorios.")
                return

            if not fecha:
                fecha = datetime.now().strftime("%d-%m-%Y")

            transaccion = {
                "tipo": "ingreso",
                "monto": monto,
                "descripcion": descripcion,
                "categoria": categoria,
                "fecha": fecha
            }
            transacciones.append(transaccion)
            guardarTransacciones()
            messagebox.showinfo("Éxito", "Ingreso registrado correctamente.")
            ingreso_ventana.destroy()

        except ValueError:
            messagebox.showerror("Error", "Monto inválido.")

    tk.Button(ingreso_ventana, text="Guardar Ingreso", command=guardar_ingreso).pack(pady=10)


# Pantalla Gastos
def registrar_gasto():
    gasto_ventana = tk.Toplevel(ventana)
    gasto_ventana.title("Registrar Gasto")
    gasto_ventana.geometry("350x300")

    balance_actual = hacerBalance()
    if balance_actual <= 0:
      messagebox.showwarning("Sin fondos", "No tienes fondos disponibles para realizar gastos.")
      gasto_ventana.destroy()
      return
    label_balance = tk.Label(gasto_ventana, text=f"Saldo actual: ${balance_actual:.2f}", font=("Arial", 12, "bold"), fg="green")
    label_balance.pack(pady=5)
    
    # Monto a retirar 
    tk.Label(gasto_ventana, text="Monto($):").pack(pady=5)
    entrada_monto = tk.Entry(gasto_ventana)
    entrada_monto.pack()
    
    # Descripción
    tk.Label(gasto_ventana, text="Descripción [opcional]:").pack(pady=5)
    entrada_descripcion = tk.Entry(gasto_ventana)
    entrada_descripcion.pack()

    # Categoría
    tk.Label(gasto_ventana, text="Categoría:").pack(pady=5)
    combo_categoria = ttk.Combobox(gasto_ventana, values=categorias, state="readonly")
    combo_categoria.pack()

    # Fecha
    tk.Label(gasto_ventana, text="Fecha (dd-mm-yyyy) [opcional]:").pack(pady=5)
    entrada_fecha = tk.Entry(gasto_ventana)
    entrada_fecha.pack()
    
    # Botón para guardar
    def guardar_gasto():
        try:
            monto = float(entrada_monto.get())
            descripcion = entrada_descripcion.get()
            categoria = combo_categoria.get()
            fecha = entrada_fecha.get().strip()

            if not categoria:
                messagebox.showwarning("Campos incompletos", "Por favor completa todos los campos obligatorios.")
                entrada_descripcion.focus()
                return

            if monto > balance_actual:
                messagebox.showerror("Fondos insuficientes", f"No tienes suficiente saldo.\nSaldo actual: ${balance_actual:.2f}")
                return

            if not fecha:
                fecha = datetime.now().strftime("%d-%m-%Y")

            transaccion = {
                "tipo": "gasto",
                "monto": monto,
                "descripcion": descripcion,
                "categoria": categoria,
                "fecha": fecha
            }
            transacciones.append(transaccion)
            guardarTransacciones()
            messagebox.showinfo("Éxito", "Gasto registrado correctamente.")
            gasto_ventana.destroy()

        except ValueError:
            messagebox.showerror("Error", "Monto inválido.")

    tk.Button(gasto_ventana, text="Guardar Gasto", command=guardar_gasto).pack(pady=10)


def ventana_historial():

    historial_ventana = tk.Toplevel()
    historial_ventana.title("Historial de Transacciones")
    historial_ventana.geometry("750x500")

    # Convertimos transacciones a DataFrame
    if not transacciones:
        messagebox.showinfo("Historial vacío", "Aún no hay transacciones registradas.")
        return

    df = pd.DataFrame(transacciones)
    df['fecha_dt'] = pd.to_datetime(df['fecha'], format="%d-%m-%Y", errors='coerce')
    df['mes'] = df['fecha_dt'].dt.strftime('%m-%Y')  # ejemplo: "07-2025"

    meses_disponibles = sorted(df['mes'].dropna().unique())

    # ------------ UI de Filtros ------------
    frame_filtros = tk.Frame(historial_ventana)
    frame_filtros.pack(pady=10)

    tk.Label(frame_filtros, text="Filtrar por mes:").grid(row=0, column=0, padx=5)

    mes_var = tk.StringVar()
    mes_dropdown = ttk.Combobox(frame_filtros, textvariable=mes_var, values=meses_disponibles, state="readonly")
    mes_dropdown.grid(row=0, column=1, padx=5)

    mostrar_todos_btn = tk.Button(frame_filtros, text="Mostrar Todos", command=lambda: actualizar_tabla(df))
    mostrar_todos_btn.grid(row=0, column=2, padx=5)

    filtrar_btn = tk.Button(frame_filtros, text="Filtrar", command=lambda: filtrar_mes(df, mes_var.get()))
    filtrar_btn.grid(row=0, column=3, padx=5)

    # ------------ Tabla Treeview ------------
    columnas = ("tipo", "monto", "descripcion", "categoria", "fecha")
    tabla = ttk.Treeview(historial_ventana, columns=columnas, show="headings")
    for col in columnas:
        tabla.heading(col, text=col.capitalize())
        tabla.column(col, width=120)
    tabla.pack(expand=True, fill="both", padx=10, pady=10)

    scrollbar = ttk.Scrollbar(tabla, orient="vertical", command=tabla.yview)
    tabla.configure(yscroll=scrollbar.set)
    scrollbar.pack(side="right", fill="y")

    def actualizar_tabla(df_filtrado):
        for item in tabla.get_children():
            tabla.delete(item)
        for _, row in df_filtrado.iterrows():
            tabla.insert("", "end", values=(
                row["tipo"],
                f"${row['monto']:.2f}",
                row["descripcion"],
                row["categoria"],
                row["fecha"]
            ))

    def filtrar_mes(df_original, mes):
        if mes:
            df_filtrado = df_original[df_original["mes"] == mes]
            actualizar_tabla(df_filtrado)

    # Mostrar todo al principio
    actualizar_tabla(df)

def ver_balance():
    messagebox.showinfo("Balance", "Función para ver balance")

# Botones del menú
boton_ingreso = tk.Button(ventana, text="Registrar Ingreso", command=registrar_ingreso)
boton_ingreso.pack(pady=5)

boton_gasto = tk.Button(ventana, text="Registrar Gasto", command=registrar_gasto)
boton_gasto.pack(pady=5)

boton_historial = tk.Button(ventana, text="Ver Historial", command=ventana_historial)
boton_historial.pack(pady=5)

boton_balance = tk.Button(ventana, text="Ver Balance", command=ver_balance)
boton_balance.pack(pady=5)

# Ejecutar la app
ventana.mainloop()

# --------------------- Manejo del archivo .JSON ---------------------

def cargarTransacciones():
  if os.path.exists("transacciones.json"):
    with open("transacciones.json", "r") as archivo:
      return json.load(archivo)
  else:
    return []

def guardarTransacciones():
  with open("transacciones.json", "w") as archivo:
    json.dump(transacciones, archivo, indent=4)

# def descargarTransacciones():
#   try:
#     files.download("transacciones.json")
#   except Exception as e:
#     print(f"Error al descargar el archivo: {e}")


# --------------------- Bienvenida ---------------------

transacciones = cargarTransacciones()
print("Bienvenido")

# --------------------- Categorias ---------------------

categorias = [
    "Alimentación",
    "Transporte",
    "Entretenimiento",
    "Salud",
    "Educación",
    "Servicios",
    "Otros"
]

# --------------------- Menús ---------------------

def showMenu():
  print("\n*** Menú Principal ***")
  print("1. Registrar Ingreso")
  print("2. Registrar Gasto")
  print("3. Ver historial de movimientos")
  print("4. Ver balance general")
  print("5. Ver estadísticas por mes")
  print("6. Ver gráficas")
  print("0. Salir")
  print("**********************")

def menuGraficas():
  print("*** Graficas ***")
  print("1. Ingresos vs Gastos")
  print("2. Distribucion de gastos")
  print("3. Balance mensual")
  print("0. Salir")
  opt = input("Seleccione una opción: ")
  match opt:
    case "1":
      graficarIngresosVsGastos()
    case "2":
      graficarGastos()
    case "3":
      graficarBalanceMensual()
    case "0":
      print("Saliendo...")
    case _:
      print("Opción inválida. Intente de nuevo.")

# --------------------- Funciones  ---------------------

def seleccionarCategoria():
    print("\nSeleccione una categoría:")
    for i, cat in enumerate(categorias, 1):
        print(f"{i}. {cat}")
    while True:
        try:
            opcion = int(input("Número de categoría: "))
            if 1 <= opcion <= len(categorias):
                return categorias[opcion - 1]
            else:
                print("Opción inválida.")
        except ValueError:
            print("Ingrese un número válido.")

def registrarIngreso():
  try:
    monto = float(input("Cuanto dinero va a ingresar? $"))
    descripcion = input("Ingrese una descripción: ")
    categoria = seleccionarCategoria()
    fecha = input("Ingrese la fecha en formato (dd-mm-yyyy) o deje en blanco para hoy: ")
    if not fecha:
      fecha = datetime.now().strftime("%d-%m-%Y")

    transaccion = {
        "tipo": "ingreso",
        "monto": monto,
        "descripcion": descripcion,
        "categoria": categoria,
        "fecha": fecha
    }
    transacciones.append(transaccion)
    guardarTransacciones()
    print("Ingreso registrado exitosamente")
  except ValueError:
    print("Monto Inválido")


def registrarGasto():
  try:
    balance = hacerBalance()
    if balance <= 0:
      print("No hay fondos suficientes para realizar el gasto")
      return
    else:
      print(f"Tu saldo actual es de: ${balance:.2f}")
    monto = float(input("Cuanto dinero va a gastar? $"))
    if monto > balance:
      print(f"Fondos insuficientes, Tu saldo actual es de: ${balance:.2f}")
      return
    descripcion = input("Ingrese una descripción: ")
    categoria = seleccionarCategoria()
    fecha = input("Ingrese la fecha (dd-mm-yyyy) o deje en blanco para hoy: ")
    if not fecha:
        fecha = datetime.now().strftime("%d-%m-%Y")
    transaccion = {
        "tipo": "gasto",
        "monto": monto,
        "descripcion": descripcion,
        "categoria": categoria,
        "fecha": fecha
    }
    transacciones.append(transaccion)
    guardarTransacciones()
    print("Gasto registrado exitosamente")
  except ValueError:
    print("Monto Inválido")

def verHistorial():
  if not transacciones:
    print("No hay movimientos registrados")
    return
  print("\n*** Historial de Movimientos ***")
  for i, t in enumerate(transacciones, 1):
    tipo = "Ingreso" if t["tipo"] == "ingreso" else "Gasto"
    print(f"{i}. {tipo} - ${t['monto']:.2f} - {t['descripcion']} - {t['categoria']} - {t['fecha']}")

def hacerBalance():
  balance = 0
  for transaccion in transacciones:
    if transaccion["tipo"] == "ingreso":
      balance += transaccion["monto"]
    else:
      balance -= transaccion["monto"]
  return balance

def verBalance():
  balance = hacerBalance()
  print(f"Tu saldo actual es de: ${balance:.2f}")

def verEstadisticas():
  return []
# --------------------- Graficas ---------------------

def graficarIngresosVsGastos():
  ingresos = sum(t["monto"] for t in transacciones if t["tipo"] == "ingreso")
  gastos = sum(t["monto"] for t in transacciones if t["tipo"] == "gasto")
  etiquetas = ['Ingresos', 'Gastos']
  valores = [ingresos, gastos]
  colores = ['green', 'red']
  plt.bar(etiquetas, valores, color=colores)
  plt.title('Ingresos vs Gastos')
  plt.ylabel('Monto ($)')
  plt.show()


def graficarGastos():
  gastos = [t for t in transacciones if t["tipo"] == "gasto"]

  if not gastos:
      print("No hay gastos para mostrar.")
      return

  # Agrupar montos por categoría
  gastos_por_categoria = defaultdict(float)
  for g in gastos:
      gastos_por_categoria[g["categoria"]] += g["monto"]

  etiquetas = list(gastos_por_categoria.keys())
  montos = list(gastos_por_categoria.values())

  plt.pie(montos, labels=etiquetas, autopct='%1.1f%%', startangle=90)
  plt.title('Distribución de gastos por categoría')
  plt.axis('equal')  # Hace que el círculo sea perfecto
  plt.show()


def graficarBalanceMensual():
  datos = defaultdict(float)

  for t in transacciones:
      mes = datetime.strptime(t["fecha"], "%d-%m-%Y").strftime("%Y-%m")
      if t["tipo"] == "ingreso":
          datos[mes] += t["monto"]
      else:
          datos[mes] -= t["monto"]
  if not datos:
      print("No hay datos suficientes.")
      return
  meses = sorted(datos)
  balances = [datos[mes] for mes in meses]
  plt.plot(meses, balances, marker='o')
  plt.title("Evolución del balance mensual")
  plt.xlabel("Mes")
  plt.ylabel("Balance ($)")
  plt.xticks(rotation=45)
  plt.grid(True)
  plt.tight_layout()
  plt.show()

# --------------------- Main ---------------------

def main():
  while True:
    showMenu()
    opcion = input("Seleccione una opción: ")
    match opcion:
      case "1":
        registrarIngreso()
      case "2":
        registrarGasto()
      case "3":
        verHistorial()
      case "4":
        verBalance()
      case "5":
        verEstadisticas()
      case "6":
        menuGraficas()
      case "0":
        print("Saliendo del programa...")
        break
      case _:
        print("Opción inválida. Intente de nuevo.")

#main()

Bienvenido
